간단한 규칙의 역할: 특정 문제에만 집중적으로 공격(엔지니어링)
조금 어거지일수도 있다는 느낌이 들수도 있음 (general하지 않고 specific)
그래서 특정 문제에만 적용가능한 점으로 제한을 둔다.
<br>
parser를 통해서 grouping하는 것까지 했다면, 해당 grouping의 정체?를 밝혀주는게 규칙기반 feature의 역할이다.
예를 들어, 부사절과 주절을 따로 grouping을 했지만 부사절의 종류(if문인지, so that문인지)를 구분하는 방법이 필요하다. 여기서 간단히 규칙칙으로 if, so that의 거리를 측정해서 이들을 구별하고자 한다. 

In [10]:
import numpy as np
import nltk
from nltk import pos_tag
from data_handlder import *

import collections
import nltk
from nltk import pos_tag
from nltk.corpus import treebank
from data_handling import *
from nltk import chunk
from nltk.tokenize.moses import MosesDetokenizer
from feature_design_for_linguistic import *
from feature_design_for_rules import *

from chunkers import ClassifierChunker
from nltk.corpus import treebank_chunk 
from chunkers import TagChunker 
train_chunks = treebank_chunk.chunked_sents()[:3000] # training data
# test_chunks = treebank_chunk.chunked_sents()[3000:] # testing data
# score = chunker.evaluate(test_chunks)
# score.accuracy() # 0.9721733155838022
chunker = ClassifierChunker(train_chunks) # a classifier trained by treebank dataset
detokenizer = MosesDetokenizer()

import os
os.environ['STANFORD_PARSER'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser.jar'
os.environ['STANFORD_MODELS'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser-3.7.0-models.jar'
from nltk.parse.stanford import StanfordParser
parser = StanfordParser(model_path='edu\\stanford\\nlp\\models\\lexparser\\englishPCFG.ser.gz')

from nltk.parse.stanford import StanfordDependencyParser
dep_parser = StanfordDependencyParser(model_path="edu\\stanford\\nlp\\models\\lexparser\\englishPCFG.ser.gz")




pre_X_train = load('pre_X_train') # preprocessed text data
pre_X_test = load('pre_X_test') # preprocessed text data

### FE2_X_train와 FE2_X_test 초기화
FE3_X_train = []
for sentence in (pre_X_train):
    temp = []
    for token in (sentence):
        temp.append([])
    FE3_X_train.append(temp)

FE3_X_test = []
for sentence in (pre_X_test):
    temp = []
    for token in (sentence):
        temp.append([])
    FE3_X_test.append(temp)

C:\Users\Office\Anaconda3\lib\site-packages\nltk\tokenize\regexp.py:123: FutureWarning: split() requires a non-empty pattern match.
  return [tok for tok in self._regexp.split(text) if tok]


In [11]:
### 일반동사가 ROOT가 되지 않는 경우들...
# be able to
# it would be
# would be nice
# it is possible
# want
# will be nice
# could be nice
# will really nice

In [12]:
li_test = 'shall search for model'
test = 'it would be nice if you be a good product.'
find_phrase = 'be nice'
test.find(find_phrase)
len(find_phrase)

import re
li_test = re.sub('search for', 'search',li_test)
print(li_test)

shall search model


In [13]:
long_word_list = 'word some one long two phrase three about above along after against'
long_word_set = set(long_word_list.split())
set('word along river'.split()) & long_word_set
#set(['along'])

{'along', 'word'}

## Simple Rule based Feature Extraction
* itwouldbenice 종류의 패턴이 있는지 없는지

In [14]:
for i, sentence in enumerate(pre_X_train): # for training data
    detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
    
    for j, token in enumerate(sentence):
        
        FE3_X_train[i][j] += check_wouldbenice_pattern(detokenized_sent)
        FE3_X_train[i][j] += is_auxVerb_leftside(j, sentence)
        FE3_X_train[i][j] += is_sothatORif_leftside(j, sentence)

        # for sub action, sub obect, ... 
        FE3_X_train[i][j] += is_causative_verb(j, sentence) # 사역동사
        FE3_X_train[i][j] += is_abstract_noun(j, sentence) # 추상명사
        
        
for i, sentence in enumerate(pre_X_test): # for training data
    detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
    
    for j, token in enumerate(sentence):
        
        FE3_X_test[i][j] += check_wouldbenice_pattern(detokenized_sent)
        FE3_X_test[i][j] += is_auxVerb_leftside(j, sentence)
        FE3_X_test[i][j] += is_sothatORif_leftside(j, sentence)
        
        # for sub action, sub obect, ... 
        FE3_X_test[i][j] += is_causative_verb(j, sentence) # 사역동사
        FE3_X_test[i][j] += is_abstract_noun(j, sentence) # 추상명사  

## Feature Extraction

In [15]:
# for i, sentence in enumerate(pre_X_train): # for training data
#     index_which = position_which(sentence)
#     index_sothat = position_sothat(sentence)
#     for j, token in enumerate(sentence):
#         #FE3_X_train[i][j] = convert_dummy((j+1) / len(sentence)) # position in a sentence
#         #FE3_X_train[i][j] = vectorized_token_using_prep_lookup(j, sentence, lookup_prep)# prep position
#         #FE3_X_train[i][j] = FE_which_left_right(j, index_which)
#         #FE3_X_train[i][j] = FE_sothat_left_right(j, index_sothat)
    
# for i, sentence in enumerate(pre_X_test): # for training data
#     index_which = position_which(sentence)
#     index_sothat = position_sothat(sentence)
#     for j, token in enumerate(sentence):
#         #FE3_X_test[i][j] = convert_dummy((j+1) / len(sentence))
#         #FE3_X_test[i][j] = vectorized_token_using_prep_lookup(j, sentence, lookup_prep)
#         #FE3_X_test[i][j] = FE_which_left_right(j, index_which)
#         #FE3_X_test[i][j] = FE_sothat_left_right(j, index_sothat)

## For Priority Class

해당 토큰이 조동사인지 아닌지...
조동사에는 다음과 같이 3가지 종류가 있는데, 우리는 두 번째 modal auxiliary verbs만 사용하기로.. 다른 것들은 중복여지가 있다.
   1. Primary auxiliary verbs
      - be / do / have
   2. Modal auxiliary verbs
      - can / could / will / would / shall / should / must / may / might
   3. Semi-modal auxiliary verbs
      - ought to / used to / need / dare

In [16]:
# auxiliary_verb_set = ['shall', 'can', 'want', 'would', 'could', 'should', 'must', 'will', 'may', 'might']

# for i, sentence in enumerate(pre_X_train): # for training data
#     for j, token in enumerate(sentence):
#         if any(token in t for t in auxiliary_verb_set):
#             FE3_X_train[i][j] = [1] # just 1 bit
#         else:
#             FE3_X_train[i][j] = [0]
            
# for i, sentence in enumerate(pre_X_test): # for training data
#     for j, token in enumerate(sentence):
#         if any(token in t for t in auxiliary_verb_set):
#             FE3_X_test[i][j] = [1]
#         else:
#             FE3_X_test[i][j] = [0]

In [17]:
# clause_set = ['that', 'which']

# for i, sentence in enumerate(pre_X_train): # for training data
#     clasue = False
#     for j, token in enumerate(sentence):
#         if any(token in t for t in clause_set):
#             clause = True
        
#         if clause == True:
#             FE3_X_train[i][j] = [1] 
#         else:
#             FE3_X_train[i][j] = [0]
            
            
# for i, sentence in enumerate(pre_X_test): # for training data
#     clasue = False
#     for j, token in enumerate(sentence):
#         if any(token in t for t in clause_set):
#             clause = True
        
#         if clause == True:
#             FE3_X_test[i][j] = [1] 
#         else:
#             FE3_X_test[i][j] = [0]

## Write out files

In [18]:
dump(FE3_X_train, 'FE3_X_train')
dump(FE3_X_test, 'FE3_X_test')
print('===> Total feature length =', len(FE3_X_train[0][0]))

===> Total feature length = 6
